In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys
import importlib

In [2]:
sys.path.append('..')

In [3]:
import modules.transforms

In [4]:
importlib.reload(modules.transforms)

<module 'modules.transforms' from '/home/jovyan/work/ETL/../modules/transforms.py'>

In [5]:
from modules.transforms import *

In [8]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [14]:
#LEER ARCHIVOS DENTRO DE RUTAS CORRESPONDIENTES A PRODUCTOS, GUARDARLOS EN DICCIONARIO "TABLES"
tables={}
for f in fs:
    files_folder=f.rsplit('/')[-1]
    files_folder_prefix='/MinCiencia/Datos-COVID19/blob/master/output/'+files_folder+'/'
    tables[files_folder]={}
    raw_path='https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/'+files_folder+'/'
    req=requests.get(f)
    files=req.text.split(files_folder_prefix)
    for idx in range(len(files)):
        files[idx]=files[idx].split('">')[0]
    files=[raw_path+f for f in files if '.csv' in f]
    for fl in files:
        try:
            print('reading file: '+fl)
            table=pd.read_csv(fl)
            table_name='_'.join(fl.rsplit('/',2)[-2:]).strip('.csv')
            tables[files_folder][table_name]=table
        except Exception as e:
            print(str(e))
            pass

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna_T.csv
reading file

In [15]:
#crear "producto11" como caso especial de producto 4
tables['producto11']={}
tables['producto11']['producto4']=pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto11/bulk/producto4.csv')
tables['producto11']['producto4'].columns=[x.replace('*','x').replace('%','porcentaje_') for x in tables['producto11']['producto4'].columns]

In [16]:
#ortables=copy.deepcopy(tables)

In [17]:
#normalizar tablas transpuestas 
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if '_T' in n[-2:]:
            print('fixing transposed table: '+n)
            tables[s][n]=transpose(tables[s][n])
#y cambiar campos "fecha" (donde haya) a tipo datetime           
for s in tables.keys():
    print('working schema: '+s)
    for n in tables[s].keys():
        if 'fecha' in [str(x).lower() for x in tables[s][n].columns]:
            print('converting to date in table: '+n)
            tables[s][n]=to_datetime(tables[s][n])
        elif 'Fecha' in [str(x).lower() for x in tables[s][n].columns]:
            print('converting to date in table: '+n)
            tables[s][n]=to_datetime(tables[s][n])

            
            
for s in tables.keys():
    print('working schema: '+s)
    for n in tables[s].keys():
        print(tables[s][n].columns)
        if 'fecha' in [str(x) for x in tables[s][n].columns]:
            print('adding reverse date and index in table: '+n)
            tables[s][n]=reverse_idx(tables[s][n],date_field='fecha')
        elif 'Fecha' in [str(x) for x in tables[s][n].columns]:
            print('adding reverse date and index in table: '+n)
            tables[s][n]=reverse_idx(tables[s][n],date_field='Fecha')
            

fixing transposed table: producto5_TotalesNacionales_T
fixing transposed table: producto14_FallecidosCumulativo_T
fixing transposed table: producto19_CasosActivosPorComuna_T
fixing transposed table: producto37_Defunciones_T
fixing transposed table: producto37_Defunciones_deis_T
fixing transposed table: producto50_DefuncionesDEISPorComuna_T
fixing transposed table: producto50_DefuncionesDEIS_confirmadosPorComuna_T
fixing transposed table: producto50_DefuncionesDEIS_sospechososPorComuna_T
fixing transposed table: producto48_SOCHIMI_T
fixing transposed table: producto24_CamasHospital_Diario_T
working schema: producto5
converting to date in table: producto5_TotalesNacionale
converting to date in table: producto5_TotalesNacionales_T
converting to date in table: producto5_TotalesNacionales_std
working schema: producto14
converting to date in table: producto14_FallecidosCumulativo_T
converting to date in table: producto14_FallecidosCumulativo_std
working schema: producto19
converting to date 

/home/jovyan/work/ETL/../modules/transforms.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['reverse_idx']=df.index.values-df.index.values.max()
/home/jovyan/work/ETL/../modules/transforms.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['MAX_DATE']=(df[date_field].max()-df[date_field]).dt.days


In [18]:
#tables['producto5']['producto5_TotalesNacionales_T']['Fecha']=pd.to_datetime(tables['producto5']['producto5_TotalesNacionales_T']['Fecha'])

In [19]:
#calcular índice y fechas invertidos para tablas en que se desea extraer último dato

#to_reverse_idx=[{'producto54':'producto54_r.nacional'},{'producto53':'producto53_confirmados_nacional'},
#                {'producto53':'producto53_confirmados_regionale'}]

#for n in to_reverse_idx:
#    k,i=list(n.items())[0]
#    print('calculating reverse index and date field in table: '+str(k)+' : '+str(i))
#    tables[k][i]=reverse_idx(tables[k][i],date_field='fecha')
    
#to_reverse_idx=[{'producto5':'producto5_TotalesNacionales_T'},
#                {'producto14':'producto14_FallecidosCumulativo_T'},
#                {'producto14':'producto14_FallecidosCumulativo_std'}]

#for n in to_reverse_idx:
#    k,i=list(n.items())[0]
#    print('calculating reverse index and date field in table: '+str(k)+' : '+str(i))
#    tables[k][i]=reverse_idx(tables[k][i],date_field='Fecha')

In [20]:
#agregar variable dummy para región metropolitana
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if 'region' in [x.lower() for x in tables[s][n].columns]:
            print('creando variable dummy para región metropolitana en tabla: '+n)
            tables[s][n]=make_dummies(tables[s][n],fields=['region'],keep_strings=['metropolitana'],value_map={'metropolitana':{0:'Resto de Chile',1:'Región Metropolitana'}})

creando variable dummy para región metropolitana en tabla: producto14_FallecidosCumulativo
creando variable dummy para región metropolitana en tabla: producto14_FallecidosCumulativo_std
creando variable dummy para región metropolitana en tabla: producto19_CasosActivosPorComuna
creando variable dummy para región metropolitana en tabla: producto19_CasosActivosPorComuna_std
creando variable dummy para región metropolitana en tabla: producto53_confirmados_provinciale
creando variable dummy para región metropolitana en tabla: producto53_confirmados_regionale
creando variable dummy para región metropolitana en tabla: producto53_confirmados_
creando variable dummy para región metropolitana en tabla: producto54_r.provincial
creando variable dummy para región metropolitana en tabla: producto54_r.provincial_copy
creando variable dummy para región metropolitana en tabla: producto54_r.provincial_n
creando variable dummy para región metropolitana en tabla: producto54_r.regional
creando variable dum

In [21]:
for k in tables.keys():
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema producto5 exists, will not be created
schema producto14 exists, will not be created
schema producto19 exists, will not be created
schema producto37 exists, will not be created
schema producto53 exists, will not be created
schema producto54 exists, will not be created
schema producto52 exists, will not be created
schema producto50 exists, will not be created
schema producto48 exists, will not be created
schema producto24 exists, will not be created
schema producto11 exists, will not be created


In [22]:
tables['producto37']['producto37_Defunciones_deis_std']

,Serie,Publicacion,Fecha,Numero defunciones,reverse_idx,MAX_DATE
0,confirmados,2020-06-18,2020-03-16,0,-449564,695.0
1,sospechosos,2020-06-18,2020-03-16,0,-449563,695.0
2,sospechosos,2020-06-28,2020-03-16,0,-449562,695.0
3,confirmados,2020-06-28,2020-03-16,0,-449561,695.0
4,sospechosos,2020-07-03,2020-03-16,0,-449560,695.0
...,...,...,...,...,...,...
449560,confirmados,2022-02-07,NaT,0,-4,NaN
449561,confirmados,2022-02-08,NaT,0,-3,NaN
449562,confirmados,2022-02-09,NaT,0,-2,NaN
449563,confirmados,2022-02-11,NaT,0,-1,NaN


In [ ]:
for schema in tables.keys():
    for name in tables[schema].keys():
        try:
            df=tables[schema][name]
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace',chunksize=50000)
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto5_TotalesNacionale ,schema: producto5
saving table/data/ETLcache/producto5_TotalesNacionale_11022022_040600.csv in cache
creating table producto5_TotalesNacionales_T ,schema: producto5
saving table/data/ETLcache/producto5_TotalesNacionales_T_11022022_040600.csv in cache
creating table producto5_TotalesNacionales_std ,schema: producto5
saving table/data/ETLcache/producto5_TotalesNacionales_std_11022022_040600.csv in cache
creating table producto14_FallecidosCumulativo ,schema: producto14
saving table/data/ETLcache/producto14_FallecidosCumulativo_11022022_040600.csv in cache
creating table producto14_FallecidosCumulativo_T ,schema: producto14
saving table/data/ETLcache/producto14_FallecidosCumulativo_T_11022022_040600.csv in cache
creating table producto14_FallecidosCumulativo_std ,schema: producto14
saving table/data/ETLcache/producto14_FallecidosCumulativo_std_11022022_040600.csv in cache
creating table producto19_CasosActivosPorComuna ,schema: producto19
saving 

In [ ]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')